In [ ]:
from lib import *


pd.set_option('display.max_columns', None)

t = CEventsTable()
t.appendFromCsv("ivan_ntuples/tH.csv",1,"tH")
t.appendFromCsv("ivan_ntuples/ttb.csv",0,"ttb")
t.appendFromCsv("ivan_ntuples/ttc.csv",0,"ttc")
t.appendFromCsv("ivan_ntuples/ttL.csv",0,"ttL")
t.appendFromCsv("nazim_ntuples/ttH.csv",0,"ttH")
t.appendFromCsv("nazim_ntuples/ttZ.csv",0,"ttZ")
t.appendFromCsv("ivan_ntuples/ttW.csv",0,"ttW")
t.appendFromCsv("ivan_ntuples/tZq.csv",0,"tZq")
t.appendFromCsv("ivan_ntuples/tWZ.csv",0,"tWZ")
t.appendFromCsv("ivan_ntuples/tW.csv",0,"single_tW")
t.appendFromCsv("ivan_ntuples/single_t1.csv",0,"single_tt")
t.appendFromCsv("ivan_ntuples/single_t2.csv",0,"single_tt")
t.appendFromCsv("ivan_ntuples/single_ts.csv",0,"single_ts")
t.appendFromCsv("nazim_ntuples/WZ.csv",0,"WZ")
t.appendFromCsv("nazim_ntuples/VV1.csv",0,"VV")
t.appendFromCsv("nazim_ntuples/VV2.csv",0,"VV")
t.appendFromCsv("ivan_ntuples/data.csv",0,"nonp")
t.subSample('ttH', 0.1)


print('Before preselection:')
t.printTypesNumbers()
t.applyPreselection()
print('After preselection:')
t.printTypesNumbers()
print('With weights:')
t.calculateWeights()
t.printTypesNumbers()



mld = CSampledDatasetAcc(t.table, (20000,100000))
mld.printInfo()



X_train = mld.X_train
y_train = mld.y_train
types_train = mld.types_train
weights_train = mld.weights_train

X_test = mld.X_test
y_test = mld.y_test
types_test = mld.types_test
weights_test = mld.weights_test


dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

treenum_grid = [50, 100, 200, 500]
depth_grid = [1,3,5,7,15]  
lr_grid = [1.0, 0.5, 0.1, 0.05, 0.01, 0.001, 0.0001]
forest_crit = "-"

TableAUCs = pd.DataFrame(columns = ['XGB', "GBDT","ADA", "RF"])
TableSignificances =  pd.DataFrame(columns = ['XGB', "GBDT","ADA", "RF"])
TableThresholds =  pd.DataFrame(columns = ['XGB', "GBDT","ADA", "RF"])
TableSignals =  pd.DataFrame(columns = ['XGB', "GBDT","ADA", "RF"])
TableBackgrounds =  pd.DataFrame(columns = ['XGB', "GBDT","ADA", "RF"])
TableTimes = pd.DataFrame(columns = ['XGB', "GBDT","ADA", "RF"])
TableF1s = pd.DataFrame(columns = ['XGB', "GBDT","ADA", "RF"])

AucCalc = CEvaluationAUC()
SigCalc = CEvaluationSignificance()

DestinationFolder = "binary_CSampledDatasetAcc"




for num_trees in treenum_grid:
    for max_depth in depth_grid:
        for lr in lr_grid:
            
            if lr == 1.0:
                forest_crit = "gini"
            elif lr == 0.5:
                forest_crit = "entropy"
            elif lr == 0.1:
                forest_crit = "log_loss"
            else:
                forest_crit = "-"
            
            desc = "nt=" + str(num_trees) + ":maxd=" + str(max_depth) + ":lr=" + str(lr) + ":crit=" +  forest_crit
            print(desc)
            
            # -----------------------------------------------------------------------------
            
            print('\nXGB:')
            model = XGBWrapper()
            model.train(dtrain, num_trees, max_depth, lr)
            preds = model.predict(X_test)
            
            auc = AucCalc.evaluate(preds, y_test)
            significance, thr, sig, bg = SigCalc.evaluate(preds, y_test, weights_test)
            f1 = calcFscore(preds, y_test, thr)
            el_time = model.elapsed_time
            
            TableAUCs.loc[desc,'XGB'] = auc
            TableSignificances.loc[desc,'XGB'] = significance
            TableThresholds.loc[desc,'XGB'] = thr
            TableSignals.loc[desc,'XGB'] = sig
            TableBackgrounds.loc[desc,'XGB'] = bg
            TableTimes.loc[desc,'XGB'] = el_time
            TableF1s.loc[desc,'XGB'] = f1
            
            print(model.elapsed_time)
            
            # -------------------------------------------------------------------------------
            
            print('\nGBDT:')
            model = SKGradientBoosting(num_trees, max_depth, lr)
            model.train(X_train, y_train)            
            preds = model.predict(X_test)
            
            auc = AucCalc.evaluate(preds, y_test)
            significance, thr, sig, bg = SigCalc.evaluate(preds, y_test, weights_test)
            f1 = calcFscore(preds, y_test, thr)
            el_time = model.elapsed_time
            
            TableAUCs.loc[desc,'GBDT'] = auc
            TableSignificances.loc[desc,'GBDT'] = significance
            TableThresholds.loc[desc,'GBDT'] = thr
            TableSignals.loc[desc,'GBDT'] = sig
            TableBackgrounds.loc[desc,'GBDT'] = bg
            TableTimes.loc[desc,'GBDT'] = el_time
            TableF1s.loc[desc,'GBDT'] = f1
            
            print(model.elapsed_time)
            
            # -------------------------------------------------------------------------------
            
            print('\nADA:')
            model = SKAdaBoost(num_trees, max_depth, lr)
            model.train(X_train, y_train)
            preds = model.predict(X_test)
            
            auc = AucCalc.evaluate(preds, y_test)
            significance, thr, sig, bg = SigCalc.evaluate(preds, y_test, weights_test)
            f1 = calcFscore(preds, y_test, thr)
            el_time = model.elapsed_time
            
            TableAUCs.loc[desc,'ADA'] = auc
            TableSignificances.loc[desc,'ADA'] = significance
            TableThresholds.loc[desc,'ADA'] = thr
            TableSignals.loc[desc,'ADA'] = sig
            TableBackgrounds.loc[desc,'ADA'] = bg
            TableTimes.loc[desc,'ADA'] = el_time
            TableF1s.loc[desc,'ADA'] = f1
            
            print(model.elapsed_time)
            
            # -----------------------------------------------------------------------------
            if forest_crit != "-":
                print('\nRF:')
                model = SKRandomForest(num_trees, max_depth, forest_crit) 
                model.train(X_train, y_train)
                preds = model.predict(X_test)

                auc = AucCalc.evaluate(preds, y_test)
                significance, thr, sig, bg = SigCalc.evaluate(preds, y_test, weights_test)
                f1 = calcFscore(preds, y_test, thr)
                el_time = model.elapsed_time

                TableAUCs.loc[desc,'RF'] = auc
                TableSignificances.loc[desc,'RF'] = significance
                TableThresholds.loc[desc,'RF'] = thr
                TableSignals.loc[desc,'RF'] = sig
                TableBackgrounds.loc[desc,'RF'] = bg
                TableTimes.loc[desc,'RF'] = el_time
                TableF1s.loc[desc,'RF'] = f1

                print(model.elapsed_time)
                
            TableAUCs.to_csv(DestinationFolder + "/aucs.csv")
            TableSignificances.to_csv(DestinationFolder + "/significances.csv")
            TableThresholds.to_csv(DestinationFolder + "/thresholds.csv")
            TableSignals.to_csv(DestinationFolder + "/signals.csv")
            TableBackgrounds.to_csv(DestinationFolder + "/backgrounds.csv")
            TableTimes.to_csv(DestinationFolder + "/times.csv")
            TableF1s.to_csv(DestinationFolder + "/f1s.csv")           
            
            print("")